In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Linear regression").getOrCreate()

In [2]:

from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
for item in data.head(5)[4]:
    print(item)
    

mstephens@davidson-herman.com
14023 Rodriguez PassagePort Jacobville, PR 37242-1057
MediumAquaMarine
33.33067252364639
12.795188551078114
37.53665330059473
4.446308318351434
599.4060920457634


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [17]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App',
                                       'Time on Website','Length of Membership'],outputCol="features")

In [18]:
output = assembler.transform(data)


In [19]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [20]:
final_data = output.select('features','Yearly Amount Spent')


In [21]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [22]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean|  498.2890529360099|
| stddev|   78.7317398344891|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [24]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean| 501.84802975052725|
| stddev|  80.95988668998807|
|    min|   266.086340948469|
|    max|  712.3963268096637|
+-------+-------------------+



In [25]:
lin_reg = LinearRegression(labelCol='Yearly Amount Spent')
lin_model = lin_reg.fit(train_data)


In [26]:
test_result = lin_model.evaluate(test_data)

In [27]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.022985342488255|
|-12.837440655198975|
| -6.554005892703401|
|0.36209585153932267|
|-18.516932916532028|
|  -5.06193116937601|
|-6.8042365071553945|
|  19.04709217390456|
|  3.848601169479366|
| -4.978155462114728|
|  1.996154350448876|
| -4.609299212988901|
|-1.3483511872585723|
|-26.927910100400027|
| -5.021103843737194|
|-19.961064056183943|
| 0.5131103404305577|
|-12.364707488788213|
|  3.905437917858876|
|-18.424122236024232|
+-------------------+
only showing top 20 rows



In [29]:
print("Mean squared error is:",test_result.rootMeanSquaredError)


Mean squared error is: 10.053565694871786


In [30]:
print("r square is:",test_result.r2)

r square is: 0.9844715927775982


In [32]:
unlabelled_data = test_data.select('features')


In [33]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8162006488763...|
|[30.8794843441274...|
|[31.0613251567161...|
|[31.3123495994443...|
|[31.3662121671876...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.5171218025062...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7656188210424...|
|[31.8164283341993...|
|[31.8293464559211...|
|[31.8648325480987...|
|[31.9480174211613...|
|[31.9563005605233...|
+--------------------+
only showing top 20 rows



In [34]:
predictions = lin_model.transform(unlabelled_data)


In [35]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.6173657301392|
|[30.3931845423455...| 332.7663104583926|
|[30.4925366965402...| 289.0252516126179|
|[30.5743636841713...|441.70231790652633|
|[30.8162006488763...|284.60327386500103|
|[30.8794843441274...| 495.2685311542307|
|[31.0613251567161...|  494.359694565057|
|[31.3123495994443...|444.54432585403606|
|[31.3662121671876...|426.74028138700555|
|[31.4252268808548...| 535.7448741168766|
|[31.4459724827577...| 482.8808105846797|
|[31.5171218025062...| 280.5277198633746|
|[31.6610498227460...| 417.7067047671594|
|[31.6739155032749...|502.65297801028123|
|[31.7656188210424...| 501.5751854793443|
|[31.8164283341993...| 521.0835555598403|
|[31.8293464559211...|384.63922764754443|
|[31.8648325480987...| 452.2559879656019|
|[31.9480174211613...|458.01543897503893|
|[31.9563005605233...|  565.550053983223|
+--------------------+------------